# problema di trasporto

Una società di logistica movimenta container vuoti da M magazzini a P porti

La richiesta dei porti (vedi tabella) può essere soddisfatta prelevando i container vuoti da uno qualsiasi dei magazzini. 

![tab1](./tab1.png)

Ogni magazzino, tuttavia, ha una disponibilità limitata di container (vedi tabella)

![tab2](./tab2.png)

La movimentazione avviene attraverso una flotta di camion, ognuno dei quali può trasportare al massimo 2 container. 

Il costo di ogni viaggio dipende dalla distanza che intercorre tra magazzino e porto ed è descritto dalla seguente tabella:

![tab3](./tab3.png)

Come devono essere organizzate le consegne al fine di minimizzare il costo totale di movimentazione?


## modello di programmazione matematica

![img1](./img1.png)

![img2](./img2.png)

In [24]:
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition

model = ConcreteModel('problema di trasporto')

i = [ 'Padova', 'Arezzo', 'Roma', 'Teramo', 'Lecce', 'Catanzaro' ]
m = [ 10, 12, 20, 24, 18, 40 ]

j = [ 'La Spezia', 'Trieste', 'Ancona', 'Napoli', 'Bari' ]
r = [ 20, 15, 25, 33, 21 ]

c = [[ 8700,  3450, 10650, 21450, 24300],
     [11400, 10200,  4950, 11400, 18300],
     [15150, 15900,  8550,  6600, 13500],
     [19650, 13500,  4650,  7200,  9450], 
     [30300, 25200, 16500,  9150,  2850], 
     [32160, 32910, 22410, 11160,  9990]]

model.i = Set(initialize=i)
model.j = Set(initialize=j)

c_dict = {}
for i, mi in enumerate(model.i):
    for j, mj in enumerate(model.j):
        c_dict[mi, mj] = c[i][j]

m = {mi: m[i] for i, mi in enumerate(model.i)}
r = {mj: r[j] for j, mj in enumerate(model.j)}

model.m = Param(model.i, initialize=m)
model.r = Param(model.j, initialize=r)
model.c = Param(model.i, model.j, initialize=c_dict)

model.x = Var(model.i, model.j, domain=NonNegativeIntegers, initialize=0)
model.y = Var(model.i, model.j, domain=NonNegativeIntegers, initialize=0)

obj_expr = sum(sum(model.c[i, j]*model.y[i, j] for j in model.j) for i in model.i)
model.cost = Objective(expr = obj_expr, sense=minimize)

model.constraints = ConstraintList()

for i in model.i:
    model.constraints.add(expr = sum(model.x[i, j] for j in model.j) <= model.m[i])

for j in model.j:
    model.constraints.add(expr = sum(model.x[i, j] for i in model.i) >= model.r[j])

for i in model.i:
    for j in model.j:
        model.constraints.add(expr = 2*model.y[i, j] >= model.x[i, j])

SolverFactory('glpk').solve(model)
model.display() 


Model problema di trasporto

  Variables:
    x : Size=30, Index=x_index
        Key                        : Lower : Value : Upper : Fixed : Stale : Domain
              ('Arezzo', 'Ancona') :     0 :   1.0 :  None : False : False : NonNegativeIntegers
                ('Arezzo', 'Bari') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
           ('Arezzo', 'La Spezia') :     0 :   6.0 :  None : False : False : NonNegativeIntegers
              ('Arezzo', 'Napoli') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
             ('Arezzo', 'Trieste') :     0 :   5.0 :  None : False : False : NonNegativeIntegers
           ('Catanzaro', 'Ancona') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
             ('Catanzaro', 'Bari') :     0 :   4.0 :  None : False : False : NonNegativeIntegers
        ('Catanzaro', 'La Spezia') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
           ('Catanzaro', 'Napoli') :     0 :  26.0 :  None : False 